## 生成科研数据variant annotation的关系数据

In [30]:
import pandas as pd
import os
import json
import re
os.getcwd()

'D:\\drug KG\\pgkb'

In [31]:
df_variant_annotation = pd.read_csv('variant_annotation/var_drug_ann.tsv', sep='\t', error_bad_lines=False).fillna("")
df_variant_annotation = df_variant_annotation[df_variant_annotation["Significance"] == "yes"]
df_variant_annotation.index = range(len(df_variant_annotation))
print(df_variant_annotation.shape)
df_variant_annotation[:2]

(5072, 11)


b'Skipping line 4308: expected 11 fields, saw 13\n'


,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population
0,1183684657,CYP2D6 ultrarapid metabolizer genotype,CYP2D6,tramadol,18204346,Metabolism/PK,yes,"Median (+)R,R-tramadol area under the curve wa...",CYP2D6 ultra-metabolizer genotype is associate...,,
1,1448997750,"CYP2B6*1, CYP2B6*18",CYP2B6,efavirenz,16495778,Metabolism/PK,yes,Please note that in the paper the allele was r...,CYP2B6 *1/*18 is associated with increased con...,*1/*18,


In [32]:
df_variant_param = pd.read_csv('variant_annotation/study_parameters.tsv', sep='\t', error_bad_lines=False).fillna("")
df_variant_param = df_variant_param[["Variant Annotation ID", "Study Type", "Study Cases", "Study Controls", "Characteristics",
                                     "Characteristics Type", "Frequency In Cases", "Frequency In Controls", "P Value", "Biogeographical Groups"]]
df_variant_merge = pd.merge(df_variant_annotation, df_variant_param, how="left", left_on="Variant Annotation ID", right_on="Variant Annotation ID")

# set 0.01 as p_value threshold to filter
p_value_list = df_variant_merge["P Value"].values
p_value_list = list(map(lambda x: "1" if ">" in str(x) else str(x).replace("<", "").replace("=", "").strip(), p_value_list))
p_value_bool_list = []
for x in p_value_list:
    try:
        if float(x) <= 0.01:
            p_value_bool_list.append(True)
        else:
            p_value_bool_list.append(False)
    except:
        p_value_bool_list.append(False)
df_variant_merge = df_variant_merge.assign(p_value_effect=p_value_bool_list)
df_variant_merge = df_variant_merge[df_variant_merge["p_value_effect"] == True]

print(df_variant_merge.shape)
df_variant_merge[:5]

(3344, 21)


,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,...,Study Type,Study Cases,Study Controls,Characteristics,Characteristics Type,Frequency In Cases,Frequency In Controls,P Value,Biogeographical Groups,p_value_effect
0,1183684657,CYP2D6 ultrarapid metabolizer genotype,CYP2D6,tramadol,18204346,Metabolism/PK,yes,"Median (+)R,R-tramadol area under the curve wa...",CYP2D6 ultra-metabolizer genotype is associate...,,...,cohort,14,,11 UM; 3 PM; healthy male non-smoking volunteers,Study Cohort,,,< 0.001,European,True
1,1448997750,"CYP2B6*1, CYP2B6*18",CYP2B6,efavirenz,16495778,Metabolism/PK,yes,Please note that in the paper the allele was r...,CYP2B6 *1/*18 is associated with increased con...,*1/*18,...,cohort,51,,,Unknown,,,< 0.0001,Multiple groups,True
2,1183684661,CYP2D6 normal metabolizer genotype,CYP2D6,tramadol,18204346,Metabolism/PK,yes,"Median (+)R,R-tramadol area under the curve wa...",CYP2D6 normal metabolizer genotype is associat...,,...,cohort,14,,11 UM; 3 PM; healthy male non-smoking volunteers,Study Cohort,,,< 0.001,European,True
3,1449191920,rs113993960,CFTR,ivacaftor / lumacaftor,28325531,Efficacy,yes,Study found a significant improvement in FEV i...,Genotype del/del is associated with response t...,del/del,...,cohort,53,,,Unknown,,,= 0.009,Unknown,True
4,1183632366,"CYP2D6*1, CYP2D6*4",CYP2D6,codeine,19940985,Metabolism/PK,yes,Lower plasma concentrations of the morphine me...,CYP2D6 *4/*4 is associated with decreased meta...,*4/*4,...,cohort,11,,End-stage renal disease patients with hemodial...,Study Cohort,,,= 0.01,Unknown,True


In [35]:
df_variant_merge.columns

Index(['Variant Annotation ID', 'Variant/Haplotypes', 'Gene', 'Drug(s)',
       'PMID', 'Phenotype Category', 'Significance', 'Notes', 'Sentence',
       'Alleles', 'Specialty Population', 'Study Type', 'Study Cases',
       'Study Controls', 'Characteristics', 'Characteristics Type',
       'Frequency In Cases', 'Frequency In Controls', 'P Value',
       'Biogeographical Groups', 'p_value_effect'],
      dtype='object')

In [20]:
list(filter(lambda x: "," in x, df_variant_merge["Variant/Haplotypes"].values))

['CYP2B6*1, CYP2B6*18',
 'CYP2D6*1, CYP2D6*4',
 'CYP3A4*1, CYP3A4*1G',
 'CYP2D6*1, CYP2D6*2, CYP2D6*2xN',
 'CYP2C9*1, CYP2C9*3',
 'CYP2A6*1A, CYP2A6*2, CYP2A6*9, CYP2A6*17',
 'UGT1A1*1, UGT1A1*28',
 'UGT1A3*1, UGT1A3*2',
 'UGT1A1*1, UGT1A1*28',
 'UGT1A1*1, UGT1A1*28',
 'UGT1A1*1, UGT1A1*28',
 'CYP3A5*1, CYP3A5*6',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*7',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3, CYP3A5*6, CYP3A5*7',
 'CYP3A5*1, CYP3A5*3, CYP3A5*6, CYP3A5*7',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'CYP3A5*1, CYP3A5*3',
 'UGT1A1*1, UGT1A1*28, UGT1A1*36, UGT1A1*37',
 'CYP2C9*1, CYP2C9*3',
 'CYP2C9*1, CYP2C9*3',
 'CYP2C9*1, CYP2C9*3',
 'CYP2C9*1, CYP2C9*3',
 'CYP2C9*1, CYP2C9*3',
 'CYP2C9*1, CYP2C9*2',
 'TPMT*1, TPMT*2, TPMT*3A, TPMT*3C, TPMT*9, TPMT*21, TPMT*33, TPMT*34',
 'TPMT*1, TPMT*2, TP

In [27]:
variant_drug_research_list = []
for index, row in df_variant_merge.iterrows():
    variant = row["Variant/Haplotypes"]
    if "genotype" in variant:
        continue
    variant_list = filter(lambda x: x!= "", [x.strip() for x in variant.split(",")])
        
    drugs = row["Drug(s)"]
    drug_list = filter(lambda x: x!= "", [x.replace("\"", "").strip() for x in re.split("/|,", drugs)])
    p_value = "P value {}".format(row["P Value"]).replace("=", "").replace("<", "")
    phenotype = row["Phenotype Category"]
    phenotype_list = set([x.replace("\"", "").strip() for x in phenotype.split(",")])
    
    biogeo_group = row["Biogeographical Groups"].replace("\"", "'")
    for d in drug_list:
        for v in variant_list:
            for p in phenotype_list:
                variant_drug_research_list.append((d, p, p_value, biogeo_group, v))

In [28]:
print(len(variant_drug_research_list))
variant_drug_research_list[:2]

4904


[('efavirenz',
  'Metabolism/PK',
  'P value  0.0001',
  'Multiple groups',
  'CYP2B6*1'),
 ('efavirenz',
  'Metabolism/PK',
  'P value  0.0001',
  'Multiple groups',
  'CYP2B6*18')]

In [29]:
pd.DataFrame(variant_drug_research_list, 
             columns=["drug", "phenotype", "p_value", "bio_geo_group", "variant"]
            ).assign(data_source=["research"] * len(variant_drug_research_list)
                    ).to_csv("processed/research_drug_variant_annotation.csv", index=False)